<a href="https://colab.research.google.com/github/CubatLin/Data-Science-Project/blob/main/Knowledge%20Graph/Gensim_TransferLearning_Pyvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author : Ethan Wu
## Data Structue
* pd_data：產品代號｜品牌｜產品名稱｜品項特性
* cus_data：客戶ID｜Attribute
* order_data: 產品代號｜客戶ID

In [1]:
import pandas as pd
pd_data =[['001','Brand_A','earphone','waterproof'],
                      ['002','Brand_A','earphone','3C'],
                      ['003','Brand_A','earphone','fashion-designed'],
                      ['004','Brand_A','mouse','3C'],
                      ['005','Brand_A','mouse','FPS'],

                      ['006','Brand_B','bikini','one-piece'],
                      ['007','Brand_B','bikini','fashion-designed'],
                      ['008','Brand_B','swimsuit','one-piece']
            ]

pd_data = pd.DataFrame(pd_data)
pd_data.columns = ['Pd_code','Brand','Product_Name','Attribute']
pd_data.head(3)

,Pd_code,Brand,Product_Name,Attribute
0,001,Brand_A,earphone,waterproof
1,002,Brand_A,earphone,3C
2,003,Brand_A,earphone,fashion-designed


In [2]:
order_data =[['001','ID_A'],
              ['002','ID_B'],
              ['003','ID_A'],
              ['004','ID_A'],
              ['005','ID_B'],
              ['006','ID_C'],
              ['007','ID_B'],
              ['008','ID_A']]

order_data = pd.DataFrame(order_data)
order_data.columns = ['Pd_code','Cust_ID']
order_data.head(3)

,Pd_code,Cust_ID
0,001,ID_A
1,002,ID_B
2,003,ID_A


In [3]:
cust_array =[['ID_A','starbucks_flg','internet_flg','drink_flg','house_flg'],
             ['ID_B','starbucks_flg','drink_flg','scooter_flg'],
             ['ID_C','bar_flg','shopping_flg'],
            ]

In [4]:
#轉換資料格式
pd_data_dic = {}
for x in pd_data.values:
  pd_data_dic[x[0]] = list(x[1:])

pd_data_dic['001']

['Brand_A', 'earphone', 'waterproof']

In [5]:
cust_data_dic = {}
for x in cust_array:
  cust_data_dic[x[0]] = x[1:]

cust_data_dic['ID_A']


['starbucks_flg', 'internet_flg', 'drink_flg', 'house_flg']

In [6]:
#Join
data_model = []

for x in order_data.values:
  data_model.append( pd_data_dic[x[0]] +cust_data_dic[x[1]])

In [7]:
data_model

[['Brand_A',
  'earphone',
  'waterproof',
  'starbucks_flg',
  'internet_flg',
  'drink_flg',
  'house_flg'],
 ['Brand_A', 'earphone', '3C', 'starbucks_flg', 'drink_flg', 'scooter_flg'],
 ['Brand_A',
  'earphone',
  'fashion-designed',
  'starbucks_flg',
  'internet_flg',
  'drink_flg',
  'house_flg'],
 ['Brand_A',
  'mouse',
  '3C',
  'starbucks_flg',
  'internet_flg',
  'drink_flg',
  'house_flg'],
 ['Brand_A', 'mouse', 'FPS', 'starbucks_flg', 'drink_flg', 'scooter_flg'],
 ['Brand_B', 'bikini', 'one-piece', 'bar_flg', 'shopping_flg'],
 ['Brand_B',
  'bikini',
  'fashion-designed',
  'starbucks_flg',
  'drink_flg',
  'scooter_flg'],
 ['Brand_B',
  'swimsuit',
  'one-piece',
  'starbucks_flg',
  'internet_flg',
  'drink_flg',
  'house_flg']]

## Transfer Learning

In [8]:
def import_data_from_drive(id):
  !wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id} -O- \
  | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt && wget --content-disposition --load-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id}'&confirm='$(<confirm.txt)

In [9]:
%%time
import_data_from_drive('1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb')

--2021-12-10 01:10:46--  https://docs.google.com/uc?export=download&id=1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb
Resolving docs.google.com (docs.google.com)... 74.125.142.138, 74.125.142.113, 74.125.142.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

-                       [ <=>                ]   3.19K  --.-KB/s    in 0s      

2021-12-10 01:10:46 (51.0 MB/s) - written to stdout [3268]

--2021-12-10 01:10:46--  https://docs.google.com/uc?export=download&id=1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb&confirm=JUka
Resolving docs.google.com (docs.google.com)... 74.125.195.139, 74.125.195.100, 74.125.195.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-4k-docs.googleusercontent.com/docs/securesc/47hot7e5i85u7s2f2b77cbhsb1cbr5o2/bdmk2c0tmp

In [10]:
!unzip CWE01.zip

Archive:  CWE01.zip
  inflating: w2v_CNA_ASBC_300d.vec   


In [11]:
import gensim
import os; os.getcwd()

'/content'

In [12]:
#準備customer字典
import itertools

cus_l = []
for x in itertools.chain(*list(cust_data_dic.values())) :
  if [x] not in cus_l:
    cus_l.append([x])

In [13]:
cus_l

[['starbucks_flg'],
 ['internet_flg'],
 ['drink_flg'],
 ['house_flg'],
 ['scooter_flg'],
 ['bar_flg'],
 ['shopping_flg']]

In [14]:
#準備prduct字典
prod_l = [[x] for x in set(pd_data.values[:,1:].flat)]
prod_l

[['swimsuit'],
 ['one-piece'],
 ['waterproof'],
 ['earphone'],
 ['fashion-designed'],
 ['Brand_B'],
 ['Brand_A'],
 ['FPS'],
 ['mouse'],
 ['3C'],
 ['bikini']]

Training

In [15]:
#定義詞向量維度
gensim_model = gensim.models.Word2Vec(min_count=1,size = 300)

In [16]:
#建立詞向量字典
gensim_model.build_vocab(cus_l+prod_l)

In [17]:
%%time

#lockf=0.0(不更新預訓練權重)/lockf=1.0(更新預訓練權重)
gensim_model.intersect_word2vec_format('/content/w2v_CNA_ASBC_300d.vec',lockf=0.6,binary=False,unicode_errors='ignore')

CPU times: user 2min 45s, sys: 3.87 s, total: 2min 49s
Wall time: 2min 44s


In [19]:
%%time
gensim_model.train(data_model,total_examples=len(data_model+cus_l+prod_l),epochs=30)

CPU times: user 31.1 ms, sys: 15.2 ms, total: 46.3 ms
Wall time: 52.8 ms


(249, 1530)

Test model & save model

In [21]:
#wordVec
gensim_model['Brand_A'][:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 2.6010952e-04, -1.1133814e-03,  1.1541219e-06, -9.2152576e-04,
       -8.6678891e-04], dtype=float32)

In [22]:
#Save Model
gensim_model.save('gensi_translearn.model')

In [23]:
#import and test
print("gensim version: ", gensim.__version__)
model_import = gensim.models.Word2Vec.load('/content/gensi_translearn.model')

gensim version:  3.6.0


In [24]:
model_import['Brand_A'][:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 2.6010952e-04, -1.1133814e-03,  1.1541219e-06, -9.2152576e-04,
       -8.6678891e-04], dtype=float32)

## Simularity Search

In [26]:
import numpy as np

#Word2Vec function
def Word2Vec_func(input_list,N,model):
  vec_attr_w2v = np.zeros(N)
  for w in input_list :
    if w in model:
      vec_attr_w2v+=model[w]
  return vec_attr_w2v

#cosine_similarity 
def w2v_similarity_func(Prod_vec,Cus_vec):
  return round( np.dot(Prod_vec,Cus_vec) / (np.linalg.norm(Prod_vec) * np.linalg.norm(Cus_vec)) ,4)

In [27]:
#將cus_l每個字轉換成詞向量
cus_attr_vec = [Word2Vec_func(x,300,gensim_model) for x in cus_l]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [40]:
pd_attr_vec = [x[1:] for x in pd_data.values]

In [41]:
#將prod每屬性轉換成詞向量
pd_attr_vec = [x[1:] for x in pd_data.values]
pd_attr_vec = [Word2Vec_func(x,300,gensim_model) for x in pd_attr_vec]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


#### 映射完詞向量，開始找各商品下最關聯的cus是誰
* 訓練邏輯: 找每個商品attribut對應到的客戶attribute最大
* 當前做法: 遍歷pd_attr_vec, 計算跟cus_attr_vec關聯，找前兩大

In [43]:
from tqdm import tqdm

prod_res_lis_1 = [];prod_res_lis_2 = []

for pd_vec in tqdm(pd_attr_vec):
  #initial
  res_1 = 0; res_label_1 = ''
  res_2 = 0; res_label_2 = ''

  for i in range(len(cus_attr_vec)):
    tmp = w2v_similarity_func(pd_vec,cus_attr_vec[i])
    if tmp>res_1:
      res_2 = res_1 ; res_1 = tmp
      res_label_2 = res_label_1 ; res_label_1 = cus_l[i][0]
    # res_2的upperbond是res_1
    if res_2<tmp<res_1:
      res_2 = tmp
      res_label_2 = cus_l[i][0]
  prod_res_lis_1.append((res_label_1,res_1))
  prod_res_lis_2.append((res_label_2,res_2))

100%|██████████| 8/8 [00:00<00:00, 1940.12it/s]


In [45]:
pd_data['sim_cus_1'] = prod_res_lis_1
pd_data['sim_cus_2'] = prod_res_lis_2
pd_data.head(5)

,Pd_code,Brand,Product_Name,Attribute,sim_cus_1,sim_cus_2
0,001,Brand_A,earphone,waterproof,"(drink_flg, 0.2134)","(starbucks_flg, 0.1513)"
1,002,Brand_A,earphone,3C,"(starbucks_flg, 0.5334)","(drink_flg, 0.4659)"
2,003,Brand_A,earphone,fashion-designed,"(drink_flg, 0.202)","(house_flg, 0.1968)"
3,004,Brand_A,mouse,3C,"(starbucks_flg, 0.5332)","(drink_flg, 0.4661)"
4,005,Brand_A,mouse,FPS,"(drink_flg, 0.1311)","(internet_flg, 0.0665)"


### Concate & PyVis

In [67]:
pd_data_1 = pd_data[pd_data.columns[:-1]]
pd_data_1.columns = ['Pd_code','Brand',	'Product_Name',	'Attribute'	,'sim_cus']

pd_data_2 = pd_data[list(pd_data.columns[:-2].values)+['sim_cus_2'] ] 
pd_data_2.columns = ['Pd_code','Brand',	'Product_Name',	'Attribute'	,'sim_cus']

In [68]:
pd_concat = pd.concat([pd_data_1 ,pd_data_2] ,axis=0 )
del pd_data_1, pd_data_2

In [75]:
pd_concat = pd_concat.reset_index(drop=True)
pd_concat['cus_seg'] = [x[0] for x in pd_concat['sim_cus']]
pd_concat['cus_seg_simscore'] = [x[1] for x in pd_concat['sim_cus']]

In [77]:
#pyvis邊上的名字一定要叫'title'

pd_concat['title'] = pd_concat['Attribute']
pd_concat.head(3)

,Pd_code,Brand,Product_Name,Attribute,sim_cus,cus_seg,cus_seg_simscore,title
0,001,Brand_A,earphone,waterproof,"(drink_flg, 0.2134)",drink_flg,0.2134,waterproof
1,002,Brand_A,earphone,3C,"(starbucks_flg, 0.5334)",starbucks_flg,0.5334,3C
2,003,Brand_A,earphone,fashion-designed,"(drink_flg, 0.202)",drink_flg,0.2020,fashion-designed


In [83]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

G = nx.from_pandas_edgelist(pd_concat,'Product_Name','cus_seg',edge_attr = 'title',
                            create_using = nx.DiGraph())

nt = Network('600px','50%',directed=True,heading = 'KG based - Grpah Vizulizaiton')
nt.from_nx(G)
nt.show_buttons(filter_ = ['physics'])

nt.show('KG based - Grpah Vizulizaiton.html')

In [84]:
import IPython
IPython.display.HTML(filename='/content/KG based - Grpah Vizulizaiton.html')